In [33]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_5 as path_palette
from bokeh.palettes import Cividis256 as contour_palette
from bokeh.models import ColumnDataSource, Toggle, Slider, Button, BuiltinIcon, CustomJS
from bokeh.layouts import column, row

import itertools

output_notebook()

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

np.random.seed(42)


Loading BokehJS ...

# **First-Order Optimizers**
<br>

**Presented by:**  
Lokesh Varadaraj  
*Entry Number:* 2022PH11858
<br>

**Course:** PYL800  
Department of Physics  
IIT Delhi

## The Optimisation Problem

In optimization, the goal is to find the best solution from a set of possible solutions. This is typically done by minimizing or maximizing an **objective function** $f(\theta)$, where $ \theta $ represents the parameters we aim to optimize. 

An optimization problem can generally be formulated as:

$$
\min_{\theta} f(\theta)
$$

Where:
- $f(\theta)$ is the **objective function**, which we want to minimize or maximize.
- $\theta$ is the vector of parameters or decision variables that influence the value of the objective function.

First-order optimization methods, aim to iteratively update the parameters $\theta$ in the direction that decreases $f(\theta)$, by utilizing the **gradient** of the objective function with respect to the parameters.


### Toy Objective Function

For the purpose of this presentation, we consider a **toy objective function** — a 2D function designed to be easily visualized. We will apply various **first-order optimisers** to find its **minima**.

The function is constructed as a sum of several Gaussian bumps, each of the form:

$$
f_i(x, y) = s_i \cdot \exp\left(-\left(a_i x + b_i y + c_i\right)^2\right)
$$

The full objective function is:

$$
L(x, y) = \sum_{i=1}^{n} f_i(x, y)
$$


In [34]:
# --- Generate loss landscape ---
def gen_gaussian():
    r = [2*(np.random.random()-0.5) for _ in range(7)]
    
    a, b, c = r[0], r[1], r[2]
    s = 10*(np.random.random()-0.6)
    
    def ax1(x, y):
        return a*x + b*y + c

    def gaussian(x, y):
        return s * np.exp(-ax1(x, y)**2)

    def gradient(x, y):
        return np.array([
            -2*a*ax1(x, y) * gaussian(x, y),
            -2*b*ax1(x, y) * gaussian(x, y)
        ])

    def hessian(x, y):
        return np.array([[
            (-2*a**2 + 4*(a*ax1(x, y))**2) * gaussian(x, y),
            (-2*a*b + 4*a*ax1(x, y)*b*ax1(x, y)) * gaussian(x, y)
        ], [
            (-2*a*b + 4*a*ax1(x, y)*b*ax1(x, y)) * gaussian(x, y),
            (-2*b**2 + 4*(b*ax1(x, y))**2) * gaussian(x, y)
        ]])

    return (gaussian, gradient, hessian)

def gen_loss_landscape(n=10):
    gaussians = [gen_gaussian() for _ in range(n)]
    
    def loss(p):
        return sum(g[0](*p) for g in gaussians)
    
    def gradient(p):
        return np.sum([g[1](*p) for g in gaussians], axis=0)
        
    def hessian(p):
        return np.sum([g[2](*p) for g in gaussians], axis=0)

    return (loss, gradient, hessian)

# --- Generate and vectorize the loss ---
L, G, H = gen_loss_landscape()
v_L = np.vectorize(L, signature='(2)->()')

# --- Grid over space ---
x = np.linspace(-10, 10, 201)
y = np.linspace(-10, 10, 201)
X, Y = np.meshgrid(x, y)
Z_input = np.stack([X, Y], axis=-1)  # Shape (201, 201, 2)
Z = v_L(Z_input)

# --- 3D Surface Plot ---
surface = go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')
fig_3d = go.Figure(data=[surface])
fig_3d.update_layout(title="Toy Objective Function (2D)", height=600, width=800)
fig_3d.show()

# --- 2D Contour Plot ---
contour = go.Contour(
    z=Z, 
    x=x, 
    y=y, 
    colorscale='Viridis',
    contours=dict(
        coloring='heatmap',
        labelfont=dict(size=12, color='white')
    ),
    colorbar=dict(
        title="Function Value"
    )
)

fig_contour = go.Figure(data=[contour])
fig_contour.update_layout(
    title="Toy Objective Function (Contours)",
    xaxis_title="x",
    yaxis_title="y",
    height=600,
    width=800
)
fig_contour.show()



## Gradient Descent

**Gradient Descent** aims to minimize an objective function by taking steps proportional to the **negative of the gradient** at the current point.

#### Update Rule:
$$\theta_{t+1} = \theta_t - \eta \nabla f(\theta_t)$$

Where:

- $\theta_t$ is the parameter vector at step $t$
- $\eta$ is the **learning rate** (step size)
- $\nabla f(\theta_t)$ is the **gradient** of the objective function at $\theta_t$

#### Intuition:

- The negative of the gradient $\nabla f(\theta)$ points in the direction of steepest **decrease** of the function.
- Gradient Descent continues until convergence to a local minimum or until a stopping criterion is met.

#### Note:
- Computing the gradient can be computationally hard and time-consuming when dealing with large datasets or high-dimensional parameter spaces.
- To make optimisation faster, we may compute the gradient using a random subset of the data (mini-batch) (Stochastic Gradient Descent - SGD).

In [ ]:
import numpy as np
import plotly.graph_objs as go
from ipywidgets import interact, FloatSlider, fixed
from numpy.linalg import norm

# --- SGD function ---
def SGD(L, G, p0, eps=1e-2,max_steps=1000,tol=5e-2):
    p = p0
    path = [p]
    velocity = [np.zeros(2)]
    g = np.inf
    while norm(g) > tol and len(path) < max_steps:
        g = G(p)
        p = p - eps*g
        path.append(p)
        velocity.append(eps*g)
    
    return np.array(path), np.array(velocity)

init_pos = np.array([-2, 3])
data = {'SGD': SGD(L, G, init_pos, eps=1e-1)}

# --- Interactive plot function ---
def plot_sgd_path(x0, y0, lr):
    init_pos = np.array([x0, y0])
    path, _ = SGD(L, G, init_pos, eps=lr)

    trace_path = go.Scatter(
        x=path[:, 0],
        y=path[:, 1],
        mode='markers+lines',
        line=dict(color='black', width=2),
        marker=dict(color='black', size=5),
        name='SGD Path'
    )

    start_point = go.Scatter(
        x=[x0],
        y=[y0],
        mode='markers',
        marker=dict(color='mediumseagreen', size=12, symbol='circle'),
        name='Start'
    )

    end_point = go.Scatter(
        x=[path[-1, 0]],
        y=[path[-1, 1]],
        mode='markers',
        marker=dict(color='darkred', size=12, symbol='x'),
        name='End'
    )

    contour = go.Contour(
        z=Z,
        x=x,
        y=y,
        colorscale='cividis',  # Subtle and clean, as used in Distill
        showscale=False,
        opacity=0.5,
        line_smoothing=0.85,
        contours=dict(showlabels=False)
    )

    fig = go.Figure(data=[contour, trace_path, start_point, end_point])
    
    # Clear the previous plot before updating
    fig.update_layout(
        title="GD Path on Toy Loss Landscape",
        xaxis_title="x",
        yaxis_title="y",
        height=600,
        width=800,
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(size=14)
    )
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)

    fig.show();

# --- Sliders ---
interact(plot_sgd_path,
         x0=FloatSlider(value=-2, min=-10, max=10, step=0.1, description='x₀'),
         y0=FloatSlider(value=3, min=-10, max=10, step=0.1, description='y₀'),
         lr=FloatSlider(value=0.1, min=0.001, max=1, step=0.01, description='Step Size'));

interactive(children=(FloatSlider(value=-2.0, description='x₀', max=10.0, min=-10.0), FloatSlider(value=3.0, d…

## Momentum

**Gradient Descent (SGD)** is simple but often slow to converge. It stops after each step, with the next step entirely determined by the local gradient. In regions with small gradients, this leads to slow progress.

**Momentum** introduces inertia — think of optimization as rolling a ball down the loss surface rather than taking discrete steps. The ball keeps moving even when the gradient is small, building speed in consistent directions.

We simulate this by defining a **velocity** term as an exponential moving average of past gradients:

$$
v_{t+1} = \gamma v_t + \nabla f(\theta_t) \\
\theta_{t+1} = \theta_t - \eta v_{t+1}
$$

- $\gamma$ is the momentum coefficient (typically $0.9$)
- $\eta$ is the learning rate

In [ ]:
import numpy as np
import plotly.graph_objs as go
from ipywidgets import interact, FloatSlider, fixed
from numpy.linalg import norm

# SGD with Momentum function
def SGD_momentum(L, G, p0, v0=None, eps=1e-2, a=0.9, grad_threshold=5e-2, max_iters=1000):
    p = p0
    v = v0 if v0 is not None else np.zeros_like(p0)
    path = [p]
    velocity = [v]
    
    g = np.inf
    iters = 0
    
    while np.linalg.norm(g) > grad_threshold and iters < max_iters:
        g = G(p)
        v = a * v - eps * g  # Momentum update
        p = p + v  # Parameter update
        path.append(p)
        velocity.append(v)
        iters += 1
    
    return np.array(path), np.array(velocity)

data['SGD_momentum'] = SGD_momentum(L, G, init_pos, eps=5e-2)

# Interactive plot function for SGD with Momentum
def plot_sgd_momentum_path(x0, y0, lr, momentum):
    init_pos = np.array([x0, y0])
    path, _ = SGD_momentum(L, G, init_pos, eps=lr, a=momentum)

    trace_path = go.Scatter(
        x=path[:, 0],
        y=path[:, 1],
        mode='markers+lines',
        line=dict(color='black', width=2),
        marker=dict(color='black', size=5),
        name='SGD with Momentum Path'
    )

    start_point = go.Scatter(
        x=[x0],
        y=[y0],
        mode='markers',
        marker=dict(color='mediumseagreen', size=12, symbol='circle'),
        name='Start'
    )

    end_point = go.Scatter(
        x=[path[-1, 0]],
        y=[path[-1, 1]],
        mode='markers',
        marker=dict(color='darkred', size=12, symbol='x'),
        name='End'
    )

    contour = go.Contour(
        z=Z,
        x=x,
        y=y,
        colorscale='cividis',  # Subtle and clean, as used in Distill
        showscale=False,
        opacity=0.5,
        line_smoothing=0.85,
        contours=dict(showlabels=False)
    )

    fig = go.Figure(data=[contour, trace_path, start_point, end_point])
    fig.update_layout(
        title="GD with Momentum Path on Toy Loss Landscape",
        xaxis_title="x",
        yaxis_title="y",
        height=600,
        width=800,
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(size=14)
    )
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.show()

# Sliders for interactive plot
interact(plot_sgd_momentum_path,
         x0=FloatSlider(value=-2, min=-10, max=10, step=0.1, description='x₀'),
         y0=FloatSlider(value=3, min=-10, max=10, step=0.1, description='y₀'),
         lr=FloatSlider(value=0.1, min=0.001, max=1, step=0.01, description='Step Size'),
         momentum=FloatSlider(value=0.9, min=0, max=1, step=0.01, description='Momentum Coefficient'));


interactive(children=(FloatSlider(value=-2.0, description='x₀', max=10.0, min=-10.0), FloatSlider(value=3.0, d…

## Nesterov Momentum

- Momentum has a key issue: once it gets rolling, it doesn't stop easily.
- This happens because the gradient added to the momentum is evaluated at the current position, not the position we would reach if we took the step.
- This isn't a problem if consecutive gradients are similar, but it becomes an issue if we "jump" across a local minimum. Momentum would push us even further forward because the gradient at the current position points downward.

Nesterov discovered this issue and proposed a fix: compute the gradient at the *future position*—the position we would be at if we took the step, assuming the gradient were zero. This "look-ahead" approach helps by "pulling" the momentum back if we overshoot a minimum, ensuring smoother convergence.

#### Update Rule
$$v_t = \gamma v_{t-1} + \eta \nabla f(\theta_{t-1} - \gamma v_{t-1})$$  
$$\theta_t = \theta_{t-1} - v_t$$

- $\gamma$: Momentum coefficient
- $\eta$: Learning rate
- $\nabla f(\theta_{t-1} - \gamma v_{t-1})$: Gradient at the "look-ahead" position

#### Key Insight
- **Standard Momentum:**  
  1. Compute gradient → Apply momentum → Update

- **Nesterov Momentum:**  
  1. Apply momentum → Compute gradient at the "look-ahead" position → Update

This method makes the optimizer more responsive, slowing down appropriately when approaching a local minimum.


In [ ]:
import numpy as np
import plotly.graph_objs as go
from ipywidgets import interact, FloatSlider, fixed
from numpy.linalg import norm

# SGD with Nesterov Momentum function
def SGD_nesterov(L, G, p0, v0=0, eps=1e-2, a=0.9, grad_threshold=5e-2, max_iters=1000):
    p = p0
    v = v0
    path = [p]
    velocity = [np.zeros_like(p0)]
    
    g = np.inf
    iters = 0
    
    while np.linalg.norm(g) > grad_threshold and iters < max_iters:
        g = G(p + a * v)  # Nesterov gradient update
        v = a * v - eps * g  # Momentum update
        p = p + v  # Parameter update
        path.append(p)
        velocity.append(v)
        iters += 1
    
    return np.array(path), np.array(velocity)

data['SGD_nesterov'] = SGD_nesterov(L, G, init_pos, eps=5e-2)

# Interactive plot function for SGD with Nesterov Momentum
def plot_sgd_nesterov_path(x0, y0, lr, momentum):
    init_pos = np.array([x0, y0])
    path, _ = SGD_nesterov(L, G, init_pos, eps=lr, a=momentum)

    trace_path = go.Scatter(
        x=path[:, 0],
        y=path[:, 1],
        mode='markers+lines',
        line=dict(color='black', width=2),
        marker=dict(color='black', size=5),
        name='Nesterov Momentum Path'
    )

    start_point = go.Scatter(
        x=[x0],
        y=[y0],
        mode='markers',
        marker=dict(color='mediumseagreen', size=12, symbol='circle'),
        name='Start'
    )

    end_point = go.Scatter(
        x=[path[-1, 0]],
        y=[path[-1, 1]],
        mode='markers',
        marker=dict(color='darkred', size=12, symbol='x'),
        name='End'
    )

    contour = go.Contour(
        z=Z,
        x=x,
        y=y,
        colorscale='cividis',  # Subtle and clean, as used in Distill
        showscale=False,
        opacity=0.5,
        line_smoothing=0.85,
        contours=dict(showlabels=False)
    )

    fig = go.Figure(data=[contour, trace_path, start_point, end_point])
    fig.update_layout(
        title="Nesterov Momentum Path on Toy Loss Landscape",
        xaxis_title="x",
        yaxis_title="y",
        height=600,
        width=800,
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(size=14)
    )
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.show()

# Sliders for interactive plot
interact(plot_sgd_nesterov_path,
         x0=FloatSlider(value=-2, min=-10, max=10, step=0.1, description='x₀'),
         y0=FloatSlider(value=3, min=-10, max=10, step=0.1, description='y₀'),
         lr=FloatSlider(value=0.1, min=0.001, max=1, step=0.01, description='Step Size'),
         momentum=FloatSlider(value=0.9, min=0, max=1, step=0.01, description='Momentum Coefficient'));


interactive(children=(FloatSlider(value=-2.0, description='x₀', max=10.0, min=-10.0), FloatSlider(value=3.0, d…

### RMSProp: Adaptive Learning Rates

#### Motivation

- While momentum helps smooth updates using past gradients, it still relies on a fixed learning rate.
- RMSProp adapts the learning rate for each parameter based on the recent magnitudes of gradients.
- This adaptation reduces oscillations and accelerates convergence, particularly in ill-conditioned problems.

#### Intuition

- Parameters with large gradients receive smaller updates.
- Parameters with small or sparse gradients receive relatively larger updates.
- This adaptive scaling leads to more stable and balanced progress during optimization.

#### Update Rule

Let:
- $g_t = \nabla f(\theta_t)$: current gradient  
- $E[g^2]_t$: running average of squared gradients  
- $\eta$: learning rate  
- $\gamma$: decay rate (typically set to $0.9$)  
- $\epsilon$: small constant to avoid division by zero  

The update equations are:

$$E[g^2]_t = \gamma E[g^2]_{t-1} + (1 - \gamma) g_t^2$$

$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{E[g^2]_t + \epsilon}} \cdot g_t$$


In [38]:

def RMSProp(L, G, p0, eps=1e-2, decay=0.9):
    
    r = np.zeros(2)
    p = p0
    v = 0
    path = [p]
    g = np.inf
    velocity = [np.zeros(2)]
    while norm(g) > 5e-2 and len(path) < 1000:
        g = G(p)
        r = decay*r + (1-decay)*g**2
        v = - ((eps)/np.sqrt(r+1e-7))*g
        p = p + v
        path.append(p)
        velocity.append(v)
    
    return np.array(path), np.array(velocity)

#data['RMSProp'] = RMSProp(L, G, init_pos, eps=1e-1)

# Adam: Adaptive Moment Estimation

**Adam** combines the strengths of **Momentum** and **RMSprop** to adapt learning rates per parameter.

## Update Rule

At each step $t$:

1. Compute gradient: $g_t = \nabla f(\theta_t)$

2. Update first moment (momentum):
   $m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t$

3. Update second moment (adaptive learning rates):
   $v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2$

4. Correct bias:
   $\hat{m}_t = \frac{m_t}{1 - \beta_1^t}$, $\hat{v}_t = \frac{v_t}{1 - \beta_2^t}$

5. Update parameters:
   $\theta_{t+1} = \theta_t - \alpha \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}$

Where:
- $\beta_1 = 0.9$ (momentum decay)
- $\beta_2 = 0.999$ (squared gradient decay)
- $\alpha$ = learning rate
- $\epsilon = 10^{-8}$ (prevents division by zero)

## Key Benefits

- **Adaptive learning rates** for each parameter
- **Momentum** to smooth updates and accelerate convergence
- **Bias correction** for better early-stage performance
- **Robust to noisy gradients** and sparse features

In [39]:
def Adam(L, G, p0, eps=1e-3, b1=0.9, b2=0.999):
    
    r = np.zeros(2)
    s = np.zeros(2)
    p = p0
    v = 0
    path = [p]
    g = np.inf
    velocity = [np.zeros(2)]
    t = 0
    while norm(g) > 5e-2 and len(path) < 1000:
        g = G(p)
        t += 1
        s = b1*s + (1-b1)*g
        r = b2*r + (1-b2)*g**2
        sh = s/(1-b1**t)
        rh = r/(1-b2**t)
        v = - ((eps)/np.sqrt(rh+1e-7))*sh
        p = p + v
        path.append(p)
        velocity.append(v)
    
    return np.array(path), np.array(velocity)

#data['Adam'] =  Adam(L, G, init_pos, eps=1e-1)

In [40]:
# Visualisations

p = figure(width=800, height=600, x_range=(-6, 1), y_range=(-2, 3))

all_data = {}
path_data = {}
paths = {}

for path in data:
    all_data[path] = {}
    all_data[path]['x'], all_data[path]['y'] = data[path][0].T
    path_data[path] = ColumnDataSource(data=dict(x=all_data[path]['x'], y=all_data[path]['y']))

colors = itertools.cycle(path_palette)
levels = np.linspace(-13,2,30)
contour_renderer = p.contour(x, y, Z, levels, line_color=contour_palette, line_alpha=0.5)

labels = [p for p in all_data]

anim_callback = CustomJS(args=dict(all_data=all_data, path_data=path_data), code="""
    const f = cb_obj.value
    for (const path in path_data) {
        const x = all_data[path].x.slice(0,f+1)
        const y = all_data[path].y.slice(0,f+1)
        path_data[path].data = {x, y}
    }
""")
slider = Slider(start=0, end=100, value=0, step=1, title="Path")
slider.js_on_change("value", anim_callback)

for path in data:
    
    color = next(colors)
    paths[path] = p.line('x', 'y', source=path_data[path], legend_label=path, 
                         color=color, line_width=2, line_alpha=0.8,
                         muted_color=color, muted_alpha=0)

toggl_js = CustomJS(args=dict(slider=slider),code="""
    var check_and_iterate = function(n_steps){
        var slider_val = slider.value;
        var toggle_val = cb_obj.active;
        if(toggle_val == false) {
            cb_obj.label = '► Play';
            clearInterval(looop);
            } 
        else if(slider_val == n_steps) {
            cb_obj.label = '► Play';
            slider.value = 0;
            cb_obj.active = false;
            clearInterval(looop);
        }
        else if(slider_val !== n_steps){
            slider.value = slider.value + 1;
        }
        else {
            clearInterval(looop);
        }
    }
    if(cb_obj.active == false){
        cb_obj.label = '► Play';
        clearInterval(looop);
    }
    else {
        cb_obj.label = '❚❚ Pause';
        var looop = setInterval(check_and_iterate, 200, 200);
    };
""")

toggl = Toggle(label='► Play',active=False)
toggl.js_on_change('active',toggl_js)

p.legend.click_policy = "mute"

full_plot = column(row(toggl, slider), p)

show(full_plot)

### Conclusions

We discussed several **first-order optimization methods**, each addressing specific challenges in gradient-based optimization:

- **Gradient Descent**: Updates parameters in the direction of the negative gradient using a fixed learning rate.
- **Momentum**: Accumulates a velocity vector to accelerate movement in consistent directions and reduce oscillations.
- **Nesterov Momentum**: Enhances momentum by computing the gradient at a look-ahead position, enabling more responsive updates and improved convergence.
- **RMSProp**: Adapts the learning rate for each parameter using a moving average of recent squared gradients, improving stability on non-stationary and ill-conditioned problems.

Other notable first-order methods include: **AdaGrad**, **Adam**, **AdaDelta**, **Nadam**, **AMSGrad**, and others.

---

### Advantages of First-Order Methods

- Require only gradient information (first derivatives), avoiding the computational cost of second-order derivatives.
- Efficient per iteration, especially in high-dimensional parameter spaces.
- Applicable to a wide range of non-linear, non-convex optimization problems.

---

### Limitations

- Slow convergence near saddle points or in flat regions where gradients vanish.
- Sensitive to hyperparameter choices like learning rate and momentum.
- May struggle on ill-conditioned loss surfaces where gradient directions are poorly aligned with the path to the optimum.

---

### Looking Ahead: Second-Order Methods

**Second-order optimization methods** use curvature information (second derivatives) for more accurate updates:

- **Newton’s Method**: Rescales gradients using the inverse Hessian for more direct convergence.
- **Quasi-Newton Methods (e.g., BFGS, L-BFGS)**: Approximate the inverse Hessian with lower computational cost.

While effective near minima, second-order methods are often expensive to compute, require significant memory, and can be impractical large datasets or high-dim parameter spaces.


### References:

- [CS231n - Optimization](https://cs231n.github.io/optimization-1/#optimization)
- [Distill - Understanding Momentum](https://distill.pub/2017/momentum/)
- [Arxiv Paper - Adam: A Method for Stochastic Optimization](https://arxiv.org/pdf/1412.6980)
- [Amaarora's Blog on Optimizers](https://amaarora.github.io/posts/2021-03-13-optimizers.html)